# Practical Introduction to LSTM Networks
#### Peter Schneider (peter.schneider@soma-analytics.com) & David Haber (david@cognitir.com) - Deep Learning Meetup - 24 June 2016 / Berlin Machine Learning Meetup, 1 August 2016

## Motivation

### Limitations of Feed-Forward Neural Networks

- Feed-forward neural networks can't make use of time information, but when modeling time series such as audio signal, stock markets or sensor signals, time information is crucial to successful modeling.
- Sequences are an integral part of intelligence. Human intelligence is based on sequential pattern recognition. Can you say the alphabet backwards?

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/46/Colored_neural_network.svg/2000px-Colored_neural_network.svg.png", width=200>

- Recurrent Neural Networks offer a way to model time information by allowing cyclical connections.

### Prominent Applications of Recurrent Neural Networks

- Speech Processing/Recognition
    - Google applies RNNs in their email auto-response technology
    - Apple uses RNNs for speech recognition in Siri
- Finance
    - Recurrent Nets have been successfully applied in automatic trading systems
- Music Composition
    - http://www.hexahedria.com/2015/08/03/composing-music-with-recurrent-neural-networks/
- Motor Control
- Biological Sequence Analysis
- Machine Translation
- Reinforcement Learning
- Meta Learning

### Backpropagation - A method to compute the gradient

* Gradient descent can help us to optimize weights in a neural network.
* To perform learning we must compute the gradient first before we can upate weights and biases
$\rightarrow$ this is what backpropagation does.
* It applies the chain rule to compute the updates layer by layer from the top to the bottom.

Let's assume following cost function for a single training example x:

$$ C = \frac{1}{2} \sum_{j} (y_j - a_j^L)^2 $$

with

$$ a^l = \sigma(z^l) $$

and 

$$ z^l \equiv w^l a^{l-1} + b^l$$

and define

$$ \delta_j^l \equiv \frac{\partial C}{\partial z_j^l}  $$

#### Error at Output Layer: $\delta^L_j$

Goal: Compute gradient of output layer

\begin{align}
\delta^L_j & = \frac{\partial C}{\partial z^L_{j}} \\
& = y_j - \sigma'(z_j^L)
\end{align}

#### Recursive Computation of Error at Hidden Layers: $\delta_j^l$

Goal: Make $\delta^l$ a function of $\delta^{l+1}$ to recursively compute gradients in lower layers

Note that

\begin{align}
z^{l+1}_k & = \sum_{m} w_{km}^{l+1} a^l_k + b_j^{l+1} \\
& = \sum_{m} w_{km}^{l+1} \sigma(z_m^l) + b_j^{l+1}
\end{align}

So to compute the full gradient of $z_j^l$ we need to gather the gradients of all higher layers

\begin{align}
\frac{\partial C}{\partial z^l_{j}} & = \sum_k \frac{\partial C}{\partial z^{l+1}_{k}} \frac{\partial z^{l+1}_{k}}{\partial z^l_{j}} \\
& = \sum_k \delta_k^{l+1} w_{jk}^{l+1} \sigma'(z_j^l)
\end{align}

the result is: $\delta_j^l = \sum_k \delta_k^{l+1} w_{jk}^{l+1} \sigma'(z_j^l)$


#### Weight Update

\begin{align} 
\frac{\partial C}{\partial w^l_{jk}} & = \frac{\partial C}{\partial z^l_{j}} \frac{\partial z_j^l}{\partial w^l_{jk}} \\
&= \delta_j^l a_k^{l-1}
\end{align}

note that $z^l_j = \sum_{k}w_{jk}a_k^{l-1} + b_j$

#### Bias Update

\begin{align} 
\frac{\partial C}{\partial b^l_{j}} & = \frac{\partial C}{\partial z^l_{j}} \frac{\partial z_j^l}{\partial b^l_{j}} \\
&= \delta_j^l
\end{align}

### Vanishing Gradient Problem - A heuristic approach

Suppose we have a network with 4 layers, a single unit per layer and use the sigmoid activation function $\sigma(x) = \frac{1}{1 + e^{-x}}$

![alternate text](single_layer_single_unit_nn.png)

Furthermore, suppose we want to compute the gradient for the bias of the first layer $\frac{\partial C}{\partial b^l_{j}}$.

First it is important to note that $\sigma'(x) = \sigma(x)(1-\sigma(x))$ is always $\leq 0.25$:

In [ ]:
import numpy as np

from bokeh.plotting import figure
from bokeh.io import show, gridplot, output_notebook

def sigmoid(x):
    return 1. / (1. + np.exp(-x)) 

p = figure(title="Activation Functions", plot_width=600, plot_height=400)
x = np.linspace(-5, 5, 100)
y_s = np.multiply(sigmoid(x),(1.-sigmoid(x)))
y_t = 1. - np.tanh(x)**2
p.line(x, y_s, color='blue', legend='first derivative of sigmoid')
p.line(x, y_t, color='green', legend='first derivative of tanh')
output_notebook()
show(p)

* Recap that $\frac{\partial C}{\partial b^l_{j}} = \delta_j^l$ so it follows from recursive substitution in $\delta^l = \frac{\partial C}{\partial a^L} \sigma'(z^L) \prod^{L-1}_{i=l} w^{i+1} \sigma'(z^i)$
* If weights are initialized to $w \sim \mathcal{N}(0,1)$ then most likely $|w| < 4$ and $\left|w \sigma'(z)\right| < 1 $
* It follows $\prod_{i=j}^{L-1} w^i \sigma'(z^i)$ will converge to zero for the lower layers in a deep neural network $\rightarrow$ so the gradients in the lower layer tend to **vanish** (keep in mind: $\sigma'(x) \leq 0.25$). 

### Exploding gradient problem

* If $\left\vert w \sigma'(z) \right\vert > 1$ then the product will explode.

### Summary

1. In general it is hard to control the vanishing and the exploding gradient problem as we would have to maintain a range of values for the input and the weights to control the gradient.

2. The problem seems to be severe especially with the sigmoid activation function (Glorot, Bengio: "Understanding the difficulty of training deep feedforward neural networks", 2010)

3. One could argue that using a tanh activation might be a better option, but keep in mind that the derivative of tanh tends to go faster to zero than the sigmoid (x > $\left\vert 1.6 \right\vert$)

### Recurrent Neural Networks

* Feedforward Neural Networks don't possess an internal state, so once an input is processed the network "forgets" about it.

* In time series processing, this is a drawback as information that is in the time structure is lost.

* Recurrent Neural Networks (RNNs) try to bridge the gap by allowing cyclical connections $$ z^{l,t}_i = w'^l_{i} a^{l-1, t} + u'^l_{i} a^{l, t-1} + b^l_i $$ and $$ a^{l,t}_i = \sigma(z^{l,t}_i) $$.

* through $ u'^l_{i} a^{l, t-1}$ (non) linear time dependencies are captured.



#### Backpropagation Through Time

* Conventional BPTT (Williams and Zipser, 1992) is similar to normal backpropagation but unrolls the neural network over time:

<img src="https://raw.githubusercontent.com/peterroelants/peterroelants.github.io/master/notebooks/RNN_implementation/img/SimpleRNN01.png", width=550>

* The time-dependent derivative is as follows: 

\begin{equation}
\delta^{t,l}_h = \sigma'(z^{l,t}_h) \left( \sum_{k=1}^{K} \delta^{t,l+1}_k w_{hk}^{l+1} + \sum_{h'=1}^{H} \delta_{h'}^{t+1,l} w_{hh'} \right)
\end{equation}

* As the weights are the same for every time step, we have to sum over them in order to get weight and bias derivative: 

\begin{equation}
\frac{\partial C}{\partial w^{l}_{i,j}} = \sum_{t=1}^{T} \frac{\partial C}{\partial z^{t,l}_j} \frac{\partial z^{t,l}_j}{\partial w^{l}_{i,j}} = \sum_{t=1}^{T} \delta_j^{t,l} \sigma(a^{l,t}_j)
\end{equation}

\begin{equation}
\frac{\partial C}{\partial b^{l}_{j}} =\sum_{t=1}^{T} \frac{\partial C}{\partial z^{t,l}_j} \frac{\partial z^{t,l}_j}{\partial b^{l}_{j}} = \sum_{t=1}^{T} \delta_j^{t,l}
\end{equation}

* Backpropagation through time exhibits the *same issues* with a vanishing or exploding gradient (Hochreiter, 1991) even for a small number of time steps (short-term memory) $\rightarrow$, therefore it is hard to capture long-range time dependencies (long-term memory).

* To circumvent this problem, Hochreiter and Schmidhuber (1997) modified recurrent units into long-short term memory units.

# Long Short Term Memory Networks (LSTMs)

- Learning to store information over extended time intervals (long-range time dependencies) via recurrent backpropagation is difficult.
- LSTMs enforces *constant* (neither exploding nor vanishing) error flow through internal states of LSTM units.
- Local in space and time! O(1) update complexity per time step and weight.
- An LSTM unit uses a set of gates to control what is entering and leaving the unit (input gate and output gate). Essentially, the unit learns when to keep, override or access information which we refer to as *cell state*.

### LSTM Unit Structure

#### Cell State (Latent State)

The LSTM is structured so that can remove or add information to the *cell state*. *Gates* regulate the information flow within the LSTM.

The flow of the *cell state* is shown in the following diagram:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-C-line.png", width=700>

#### Forget Gate

The *forget gate* applies a sigmoid function to the the previous output $h_{t-1}$ and input $x_t$. It outputs a number between 0 and 1 to decide which part of the previous *cell state* it wants to keep. Zero means "don't let anything through", one means "let everything through".

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-f.png", width=700>

#### Input Gate

The *input gate* controls the flow of new information into the new *cell state*. A *sigmoid layer* regulates which values we will update and a *tanh layer* proposes values that could be added to the state.

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-i.png", width=700>

#### Cell State Update

We update step of the *cell state* is simple. We multiply the old state $C_{t-1}$ by the output of the *forget layer* $f_t$ and add $i_t * \tilde{C}_t$:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-C.png", width=700>

#### Output Gate

As a last step, we generate the LSTM unit's output:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-focus-o.png", width=700>

#### LSTM Modules

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png", width=700>

### LSTM Structure Variants

* Peephole connections (Gers & Schmidhuber, 2000)
* Gated Recurrent Units (Cho, et al., 2014)
* Depth Gated RNNs (Yao, et al., 2015)
* Clockwork RNNs (Koutnik, et al., 2014)
* Highway Networks (Srivastava, Greff, Schmidhuber, 2015)

See "LSTM: A search space odyssey" (Greff, et al., 2015) for a useful comparison of popular variants.

### Keras

Keras is a neural networks library that can run on either Theano or TensorFlow. Keras has been build for:

- Modularity
- Minimalism
- Easy extensibility
- Work with Python

### Keras Implementation of an LSTM

#### Import Libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing import sequence

from pandas import read_csv
from sklearn.cross_validation import train_test_split
from scipy import stats
from pandas_datareader.data import DataReader

import datetime

import numpy as np
np.random.seed(25)

import matplotlib.pyplot as plt
%matplotlib inline

import time

#### Load Data

In [ ]:
start = datetime.datetime(1962, 1, 1)
end = datetime.datetime(2016, 1, 1)

DataReader("GE", 'yahoo', start, end).to_csv("ge.csv")

In [ ]:
timesteps = 5
slide = 3
prediction_steps = 1

# Load Data
df = read_csv('ge.csv')

# series[0] is open price at 1.1.1962
series = df['Open'].as_matrix()

# Standardize data
series = (series - np.mean(series)) / np.std(series)

# Create starting indices window
a = np.arange(0, len(series)-(timesteps+prediction_steps), slide)

# Create window summation indices
b = np.arange(0, timesteps + prediction_steps)

# Create mask
start_tiles = np.tile(a, (timesteps + prediction_steps, 1)).T
sum_tiles = np.tile(b, (len(start_tiles), 1))
index_mask = start_tiles + sum_tiles
x_mask = index_mask[:,:-prediction_steps]
y_mask = index_mask[:, -prediction_steps:]

# Build data set
x, y = series[x_mask], series[y_mask]

# Split data
split_idx = int(len(x) * 0.75)

x_train = x[:split_idx]
y_train = y[:split_idx]

x_test = x[split_idx:]
y_test = y[split_idx:]

# Need to add one more dim to make batches
x_train = np.expand_dims(x_train, 2)
x_test = np.expand_dims(x_test, 2)

print x_train.shape
print y_train.shape

#### Define Model Architecture

In [ ]:
model = Sequential()
model.add(LSTM(25, input_shape=(timesteps, 1), activation='sigmoid'))#, return_sequences=True))
#model.add(LSTM(25, input_shape=(timesteps, 1), activation='sigmoid'))
model.add(Dense(prediction_steps, activation='linear'))

#### Configure Learning Process

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss='mse', optimizer='rmsprop')

print model.summary()

#### Perform Training in Batches

In [ ]:
hist = model.fit(x_train, y_train, batch_size=batch_size, nb_epoch=epochs, verbose=0)
plt.figure(figsize=(35,10))
plt.plot(np.arange(0, epochs), hist.history['loss'])
plt.show()

#### Evaluate Performance

In [ ]:
# Evaluate model
y_pred = model.predict(x_test, batch_size=batch_size)

for i in range(prediction_steps):
    rmse = np.sqrt(np.sum((y_test[:, i] - y_pred[:, i])**2))
    print "LSTM RMSE %i: %.4f" % (i, rmse)

#### Visualize Performance

In [ ]:
def chow(X, Y, breakpoint, alpha = 0.05):
    """
    Performs a chow test.
    Split input matrix and output vector into two
    using specified breakpoint.
     X - independent variables matrix
     Y - dependent variable vector
     breakpoint - index to split.
     alpha is significance level for hypothesis test
    """
    k = len(X[0])
    n = len(X)

    # Split into two datasets.
    X1 = X[:breakpoint][:]
    Y1 = Y[:breakpoint][:]

    #print X1.shape
    #print Y1.shape

    X2 = X[breakpoint:][:]
    Y2 = Y[breakpoint:][:]
    
    #print X2.shape
    #print Y2.shape

    # Perform separate three least squares.
    #allfit   = lm.ols(X,Y)
    allfit = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), Y)
    #lowerfit = lm.ols(X1, Y1)
    lowerfit = np.dot(np.dot(np.linalg.inv(np.dot(X1.T, X1)), X1.T), Y1)
    #upperfit = lm.ols(X2, Y2)
    upperfit = np.dot(np.dot(np.linalg.inv(np.dot(X2.T, X2)), X2.T), Y2)

    RSS  = np.sum(np.square(np.dot(X, allfit) - Y)) #allfit.RSS
    RSS1 = np.sum(np.square(np.dot(X1, lowerfit) - Y1)) #lowerfit.RSS
    RSS2 = np.sum(np.square(np.dot(X2, upperfit) - Y2)) #upperfit.RSS

    df1 = k
    df2 = n - 2 *k

    num = (RSS - (RSS1 + RSS2)) / float(df1)
    den = (RSS1 + RSS2) / (df2)

    Ftest = num/den
    Fcrit = stats.f.ppf([1 -0.05], df1, df2)
    return (Ftest, Fcrit, df1, df2, RSS, RSS1, RSS2)

t = np.arange(0, len(y_pred[:1000, 0]))

plt.figure(figsize=(35, 15))
plt.plot(t, y_test.T.ravel()[:1000], 'b')
plt.plot(t, y_pred.T.ravel()[:1000], 'r')
plt.show()

lower = 5
upper = x_test.shape[0] - 2
idx = lower
Ftests = []
Fcrits = []
x_chow = x_test[:, :, 0]
y_chow = y_test[:, 0]

while idx <= upper:
    (Ftest, Fcrit, df1, df2, RSS, RSS1, RSS2) = chow(x_chow, y_chow, idx)
    Ftests.append(Ftest)
    Fcrits.append(Fcrit[0])
    idx += 1

plt.figure(figsize=(35, 15))
plt.plot(Ftests)
plt.plot(Fcrits)
plt.ylim([-5, 10])
plt.show()

### Impulse Response of Network

In [ ]:
from collections import defaultdict

def random_color():
    rgb = []
    for i in range(3):
        c = np.random.randint(0, 256, 1)
        rgb += [c[0]]

    return '#%02x%02x%02x' % tuple(rgb)

'''
Setting the activations functions
'''
# gate activation function (tanh)
f = np.tanh
# cell activation function (sigmoid)
g = sigmoid
# unit output activation
h = np.tanh
'''
Initialize the plots
'''
width, height = 500, 300
p_data = figure(title="Input", plot_width=width, plot_height=height)
p_unit = figure(title="Unit output", plot_width=width, plot_height=height)
p_out = figure(title="Output Gate", plot_width=width, plot_height=height)
p_cell = figure(title="Cell State", plot_width=width, plot_height=height)
p_forget = figure(title="Forget Gate", plot_width=width, plot_height=height)
p_input = figure(title="Input Gate", plot_width=width, plot_height=height)
'''
Initialize loop
'''
repeat = 20
T = 30
time = np.arange(0, T)
weight_dict = defaultdict(list)
for _ in range(repeat):
    '''
    Initializing weights and states
    '''
    # unit input weights
    w_i, w_f, w_c, w_o = np.random.standard_normal(4)
    # recurrent connections
    u_i, u_f, u_c, u_o = np.random.standard_normal(4)
    # peephole connections
    c_i, c_f, c_o = np.random.standard_normal(3)
    # store input weights
    weight_dict['w_i'] += [w_i]
    weight_dict['w_f'] += [w_f]
    weight_dict['w_c'] += [w_c]
    weight_dict['w_o'] += [w_o]
    # store recurrent conns
    weight_dict['u_i'] += [u_i]
    weight_dict['u_f'] += [u_f]
    weight_dict['u_c'] += [u_c]
    weight_dict['u_o'] += [u_o]
    # store peephole connections
    weight_dict['c_i'] += [c_i]
    weight_dict['c_f'] += [c_f]
    weight_dict['c_o'] += [c_o]    
    '''
    Simlate cell over multiple time steps with a spike at t=0
    '''
    a_i, a_f, a_o = np.zeros(T), np.zeros(T), np.zeros(T)
    s_c, a_b = np.zeros(T+1), np.zeros(T+1)
    x = np.zeros(T)
    x[0] = 1
    for m, t in zip(range(T), range(1, T+1)):
        # input gate
        z_i = w_i * x[m] + u_i * a_b[t-1] + c_i * s_c[t-1]
        a_i[m] = f(z_i)
        # forget gate
        z_f = w_f * x[m] + u_f * a_b[t-1] + c_f * s_c[t-1]
        a_f[m] = f(z_f)
        # cell state
        z_c = w_c * x[m] + u_c * a_b[t-1]
        s_c[t] = a_f[m] * s_c[t-1] + a_i[m] * g(z_c)
        # output gate
        z_o = w_o * x[m] + u_o * a_b[t-1] + c_o * s_c[t]
        a_o[m] = f(z_o)
        # unit output
        a_b[t] = a_o[m] * h(s_c[t])

    # delete the initial values
    s_c = s_c[1:]
    a_b = a_b[1:]
    # store final values
    weight_dict['a_i'] += [a_i[-1]]
    weight_dict['a_f'] += [a_f[-1]]
    weight_dict['a_o'] += [a_o[-1]]
    weight_dict['a_b'] += [a_b[-1]]
    weight_dict['s_c'] += [s_c[-1]]
    # plot
    p_data.line(time, x, color=random_color())
    p_unit.line(time, a_b, color=random_color())
    p_out.line(time, a_o, color=random_color())
    p_cell.line(time, s_c, color=random_color())
    p_forget.line(time, a_f, color=random_color())
    p_input.line(time, a_i, color=random_color())
    
# show the plotq
grid = [[p_data, p_input], [p_forget, p_cell], [p_out, p_unit]]
show(gridplot(grid))

### Sources

* RNN and LSTM images taken from http://colah.github.io/posts/2015-08-Understanding-LSTMs/
* Supervised Sequence Labelling with Recurrent Neural Networks (Graves): https://www.cs.toronto.edu/~graves/preprint.pdf
* Neural Networks and Deep Learning: http://www.neuralnetworksanddeeplearning.com